# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [33]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps.datasets

# Import API key
from api_keys import g_key

ModuleNotFoundError: No module named 'gmaps'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_path='city_data.csv'

In [3]:
weather_data=pd.read_csv(weather_path)
weather_data.head()


,Unnamed: 0,City,Country,Longitude (Degrees),Latitude (Degrees),Maximum Temperature (C),Humidity (%),Wind Speed (km/h),Cloudiness (%)
0,0,São Filipe,CV,-24.50,14.90,27.74,70,4.04,2
1,1,Rikitea,PF,-134.97,-23.12,23.34,79,1.48,41
2,2,Berlevåg,NO,29.09,70.86,2.28,86,17.50,75
3,3,New Norfolk,AU,147.06,-42.78,11.39,79,0.46,100
4,4,Whittlesea,ZA,26.82,-32.18,28.40,26,15.24,99


In [4]:
weather_data.rename({'Longitude (Degrees)': 'Lng', 'Latitude (Degrees)': 'Lat'}, axis=1, inplace=True)

In [5]:
type(weather_data)

pandas.core.frame.DataFrame

In [6]:
weather_data.shape

(568, 9)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [97]:
gmaps.configure(api_key=g_key)

NameError: name 'gmaps' is not defined

In [7]:
locations = weather_data[["Lat", "Lng"]].astype(float)
humidity = weather_data["Humidity (%)"].astype(float)

In [8]:
type(locations)

pandas.core.frame.DataFrame

In [9]:
locations.head()

,Lat,Lng
0,14.90,-24.50
1,-23.12,-134.97
2,70.86,29.09
3,-42.78,147.06
4,-32.18,26.82


In [101]:
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)
fig.add_layer(heatmap_layer)

fig


NameError: name 'gmaps' is not defined

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
type(weather_data)

pandas.core.frame.DataFrame

In [11]:
hotel_df=weather_data.dropna()

In [12]:
type(hotel_df)

pandas.core.frame.DataFrame

In [13]:
hotel_df.head()

,Unnamed: 0,City,Country,Lng,Lat,Maximum Temperature (C),Humidity (%),Wind Speed (km/h),Cloudiness (%)
0,0,São Filipe,CV,-24.50,14.90,27.74,70,4.04,2
1,1,Rikitea,PF,-134.97,-23.12,23.34,79,1.48,41
2,2,Berlevåg,NO,29.09,70.86,2.28,86,17.50,75
3,3,New Norfolk,AU,147.06,-42.78,11.39,79,0.46,100
4,4,Whittlesea,ZA,26.82,-32.18,28.40,26,15.24,99


In [14]:
type(hotel_df)

pandas.core.frame.DataFrame

In [15]:
hotel_df=hotel_df[hotel_df["Maximum Temperature (C)"] >20]

In [16]:
hotel_df=hotel_df[hotel_df["Wind Speed (km/h)"] <10]


In [17]:
hotel_df=hotel_df[hotel_df["Humidity (%)"] <70]

In [18]:
hotel_df=hotel_df[hotel_df["Maximum Temperature (C)"] <30]

In [19]:
hotel_df=hotel_df[hotel_df["Cloudiness (%)"] <50]

In [20]:
hotel_df=hotel_df[hotel_df["Humidity (%)"] >50]

In [21]:
hotel_df.shape

(37, 9)

In [22]:
type(hotel_df)

pandas.core.frame.DataFrame

In [23]:
hotel_df.shape

(37, 9)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
hotel_df['Hotel'] = ""

In [25]:
hotel_df.head()

,Unnamed: 0,City,Country,Lng,Lat,Maximum Temperature (C),Humidity (%),Wind Speed (km/h),Cloudiness (%),Hotel
7,7,Kruisfontein,ZA,24.73,-34.00,23.89,69,4.02,0,
18,18,Hermanus,ZA,19.23,-34.42,24.44,69,2.24,21,
22,22,Kahului,US,-156.47,20.89,22.11,68,1.58,1,
35,35,Ribeira Grande,PT,-28.70,38.52,21.00,64,4.60,40,
41,41,Port Alfred,ZA,26.89,-33.59,25.25,69,7.83,47,


In [26]:
hotel_df.dtypes

Unnamed: 0                   int64
City                        object
Country                     object
Lng                        float64
Lat                        float64
Maximum Temperature (C)    float64
Humidity (%)                 int64
Wind Speed (km/h)          float64
Cloudiness (%)               int64
Hotel                       object
dtype: object

In [27]:
# find the closest hotel for each set of coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
  
 

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city= row["City"]

    params = {
    "location": f"{lat}, {lng}",
    "radius": 5000,
    'keyword': "Hotel",
    "key": g_key,
        }  

# assemble url and make API request
    print(f"Retrieving Results for {city} at coordinates ({lat}, {lng})")
    response = requests.get(base_url, params=params).json()
    
# extract results
    results = response['results']
    
    try:
        print(f"Closest hotel in {city} is {results[0]['name']}.")
        print("------------")
        hotel_df.loc[index, "Hotel"] = results[0]["name"]


    except :
        print("Missing field/result... skipping.")
        print("------------")

Retrieving Results for Kruisfontein at coordinates (-34.0, 24.73)
Closest hotel in Kruisfontein is Humansdorp Boutique Hotel.
------------
Retrieving Results for Hermanus at coordinates (-34.42, 19.23)
Closest hotel in Hermanus is Harbour House Hotel.
------------
Retrieving Results for Kahului at coordinates (20.89, -156.47)
Closest hotel in Kahului is Maui Beach Hotel.
------------
Retrieving Results for Ribeira Grande at coordinates (38.52, -28.7)
Closest hotel in Ribeira Grande is Quinta Das Buganvílias.
------------
Retrieving Results for Port Alfred at coordinates (-33.59, 26.89)
Closest hotel in Port Alfred is MyPond Hotel.
------------
Retrieving Results for Richards Bay at coordinates (-28.78, 32.04)
Closest hotel in Richards Bay is BON Hotel Waterfront Richards Bay.
------------
Retrieving Results for Arraial do Cabo at coordinates (-22.97, -42.03)
Closest hotel in Arraial do Cabo is Orlanova Hotel.
------------
Retrieving Results for Grand Gaube at coordinates (-20.01, 57.66

In [28]:
hotel_df.head()

,Unnamed: 0,City,Country,Lng,Lat,Maximum Temperature (C),Humidity (%),Wind Speed (km/h),Cloudiness (%),Hotel
7,7,Kruisfontein,ZA,24.73,-34.00,23.89,69,4.02,0,Humansdorp Boutique Hotel
18,18,Hermanus,ZA,19.23,-34.42,24.44,69,2.24,21,Harbour House Hotel
22,22,Kahului,US,-156.47,20.89,22.11,68,1.58,1,Maui Beach Hotel
35,35,Ribeira Grande,PT,-28.70,38.52,21.00,64,4.60,40,Quinta Das Buganvílias
41,41,Port Alfred,ZA,26.89,-33.59,25.25,69,7.83,47,MyPond Hotel


In [29]:
hotel_df.shape

(37, 10)

In [30]:
#Filter out rows where we did not find a hotel name.

In [31]:
hotel_df=hotel_df[hotel_df["Hotel"] !=""]

In [32]:
hotel_df.shape

(35, 10)

In [34]:
hotel_df=hotel_df[:10]

In [35]:
hotel_df.shape

(10, 10)

In [37]:
type(hotel_df)

pandas.core.frame.DataFrame

In [36]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [37]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()])
fig.add_layer(markers)


In [38]:
fig

Figure(layout=FigureLayout(height='420px'))